In [0]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split

In [0]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from model.mask_rcnn import maskrcnn_resnet50_fpn, MaskRCNNPredictor

def get_model(num_classes):
    model = maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [0]:
CLASSES = [
        "__background__ ",
        "aeroplane",
        "bicycle",
        "bird",
        "boat",
        "bottle",
        "bus",
        "car",
        "cat",
        "chair",
        "cow",
        "diningtable",
        "dog",
        "horse",
        "motorbike",
        "person",
        "pottedplant",
        "sheep",
        "sofa",
        "train",
        "tvmonitor",
]

In [0]:
import numpy as np

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
import transforms as T
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    # transforms.append(PrepareItem())
    
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)
  

In [0]:
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import download_url, check_integrity, verify_str_arg
import os

In [0]:
import xml.etree.ElementTree as ET
import collections
from PIL import Image

In [0]:
DATASET_YEAR_DICT = {
    '2012': {
        'url': 'http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar',
        'filename': 'VOCtrainval_11-May-2012.tar',
        'md5': '6cd6e144f989b92b3379bac3b3de84fd',
        'base_dir': os.path.join('VOCdevkit', 'VOC2012')
    }
}

In [0]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar 
!mkdir VOC2012
!tar xvf /content/VOCtrainval_11-May-2012.tar -C /content/VOC2012

In [0]:
class VOCSegmentation(VisionDataset):
    """`Pascal VOC <http://host.robots.ox.ac.uk/pascal/VOC/>`_ Segmentation Dataset.

    Args:
        root (string): Root directory of the VOC Dataset.
        year (string, optional): The dataset year, supports years 2007 to 2012.
        image_set (string, optional): Select the image_set to use, ``train``, ``trainval`` or ``val``
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.
    """

    def __init__(self,
                 root,
                 year='2012',
                 image_set='train',
                 download=False,
                 transform=None,
                 target_transform=None,
                 transforms=None):
        super(VOCSegmentation, self).__init__(root, transforms, transform, target_transform)
        self.year = year
        self.url = DATASET_YEAR_DICT[year]['url']
        self.filename = DATASET_YEAR_DICT[year]['filename']
        self.md5 = DATASET_YEAR_DICT[year]['md5']
        valid_sets = ["train", "trainval", "val"]
        if year == "2007":
            valid_sets.append("test")
        self.image_set = verify_str_arg(image_set, "image_set", valid_sets)
        base_dir = DATASET_YEAR_DICT[year]['base_dir']
        voc_root = os.path.join(self.root, base_dir)
        image_dir = os.path.join(voc_root, 'JPEGImages')
        mask_dir = os.path.join(voc_root, 'SegmentationObject')
        annotation_dir = os.path.join(voc_root, 'Annotations')

        if download:
            download_extract(self.url, self.root, self.filename, self.md5)

        if not os.path.isdir(voc_root):
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        splits_dir = os.path.join(voc_root, 'ImageSets/Segmentation')

        split_f = os.path.join(splits_dir, image_set.rstrip('\n') + '.txt')

        with open(os.path.join(split_f), "r") as f:
            file_names = [x.strip() for x in f.readlines()]

        self.images = [os.path.join(image_dir, x + ".jpg") for x in file_names]
        self.masks = [os.path.join(mask_dir, x + ".png") for x in file_names]
        self.annotations = [os.path.join(annotation_dir, x + ".xml") for x in file_names]
        assert (len(self.images) == len(self.masks))

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is the image segmentation.
        """
        img = Image.open(self.images[index]).convert('RGB')
        mask = Image.open(self.masks[index])
        mask = np.array(mask)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]
        tgt = self.parse_voc_xml(
            ET.parse(self.annotations[index]).getroot()
        )
        bboxes = tgt['annotation']['object']
        if len(obj_ids) > 1:
          if type(bboxes) == list and len(bboxes) != len(obj_ids):
            obj_ids = obj_ids[:-1]
          if type(bboxes) == dict:
            obj_ids = obj_ids[:-1]
        num_objs = len(obj_ids)
        masks = mask == obj_ids[:, None, None]
        boxes = []
        labels = []
        # plt.imshow(mask)
        if num_objs == 1:
          boxes.append([int(bboxes['bndbox']['xmin']),
                          int(bboxes['bndbox']['ymin']),
                          int(bboxes['bndbox']['xmax']),
                          int(bboxes['bndbox']['ymax'])])
          labels.append(CLASSES.index(bboxes['name']))
        else:
          for i in range(num_objs):
            boxes.append([int(bboxes[i]['bndbox']['xmin']),
                          int(bboxes[i]['bndbox']['ymin']),
                          int(bboxes[i]['bndbox']['xmax']),
                          int(bboxes[i]['bndbox']['ymax'])])
            labels.append(CLASSES.index(bboxes[i]['name']))
          
        image_id = torch.tensor([index])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target

    def __len__(self):
        return len(self.images)

    def parse_voc_xml(self, node):
        voc_dict = {}
        children = list(node)
        if children:
            def_dic = collections.defaultdict(list)
            for dc in map(self.parse_voc_xml, children):
                for ind, v in dc.items():
                    def_dic[ind].append(v)
            voc_dict = {
                node.tag:
                    {ind: v[0] if len(v) == 1 else v
                     for ind, v in def_dic.items()}
            }
        if node.text:
            text = node.text.strip()
            if not children:
                voc_dict[node.tag] = text
        return voc_dict

In [0]:
dataset = VOCSegmentation("VOC2012", 
                                            year='2012', 
                                            transforms=get_transform(train=True),
                                            download=False)
dataset_test = VOCSegmentation("VOC2012", 
                                            year='2012', image_set='val', 
                                            transforms=get_transform(train=False),
                                            download=False)

In [0]:
dataset[2]

(tensor([[[0.8392, 0.9294, 1.0000,  ..., 1.0000, 1.0000, 0.6784],
          [0.9686, 1.0000, 1.0000,  ..., 0.9843, 1.0000, 0.4863],
          [0.9961, 0.9725, 0.9725,  ..., 0.9725, 1.0000, 0.2196],
          ...,
          [0.3294, 0.3529, 0.3686,  ..., 0.4118, 0.2471, 0.2510],
          [0.3216, 0.2118, 0.3216,  ..., 0.2157, 0.2706, 0.2667],
          [0.2118, 0.3490, 0.2471,  ..., 0.3137, 0.1686, 0.2627]],
 
         [[0.8314, 0.9216, 1.0000,  ..., 0.9608, 0.9647, 0.6000],
          [0.9569, 0.9961, 0.9882,  ..., 0.9569, 0.9765, 0.4118],
          [0.9922, 0.9686, 0.9686,  ..., 0.9725, 0.9922, 0.1451],
          ...,
          [0.3569, 0.3882, 0.4118,  ..., 0.4431, 0.2902, 0.3059],
          [0.3765, 0.2745, 0.3843,  ..., 0.2471, 0.3216, 0.3373],
          [0.2549, 0.3961, 0.2941,  ..., 0.3765, 0.2353, 0.3412]],
 
         [[0.7843, 0.8745, 0.9569,  ..., 0.9608, 0.9490, 0.5569],
          [0.9294, 0.9686, 0.9608,  ..., 0.9255, 0.9333, 0.3529],
          [0.9765, 0.9529, 0.9529,  ...,

In [0]:
#1012, 1382 814
for i in range(len(dataset_test)):
  print(".", end="")
  try:
    temp = dataset[i]
  except Exception as e:
    print(i)
    print(e)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [0]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=collate_fn)

In [0]:
len(data_loader)

732

In [0]:
######### From the TorchVision Tutorial ##########
from collections import defaultdict, deque
import datetime
import pickle
import time
import torch.distributed as dist
import errno


class SmoothedValue(object):
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device='cuda')
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value)


def all_gather(data):
    """
    Run all_gather on arbitrary picklable data (not necessarily tensors)
    Args:
        data: any picklable object
    Returns:
        list[data]: list of data gathered from each rank
    """
    world_size = get_world_size()
    if world_size == 1:
        return [data]

    # serialized to a Tensor
    buffer = pickle.dumps(data)
    storage = torch.ByteStorage.from_buffer(buffer)
    tensor = torch.ByteTensor(storage).to("cuda")

    # obtain Tensor size of each rank
    local_size = torch.tensor([tensor.numel()], device="cuda")
    size_list = [torch.tensor([0], device="cuda") for _ in range(world_size)]
    dist.all_gather(size_list, local_size)
    size_list = [int(size.item()) for size in size_list]
    max_size = max(size_list)

    # receiving Tensor from all ranks
    # we pad the tensor because torch all_gather does not support
    # gathering tensors of different shapes
    tensor_list = []
    for _ in size_list:
        tensor_list.append(torch.empty((max_size,), dtype=torch.uint8, device="cuda"))
    if local_size != max_size:
        padding = torch.empty(size=(max_size - local_size,), dtype=torch.uint8, device="cuda")
        tensor = torch.cat((tensor, padding), dim=0)
    dist.all_gather(tensor_list, tensor)

    data_list = []
    for size, tensor in zip(size_list, tensor_list):
        buffer = tensor.cpu().numpy().tobytes()[:size]
        data_list.append(pickle.loads(buffer))

    return data_list


def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict


class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError("'{}' object has no attribute '{}'".format(
            type(self).__name__, attr))

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(
                "{}: {}".format(name, str(meter))
            )
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ''
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt='{avg:.4f}')
        data_time = SmoothedValue(fmt='{avg:.4f}')
        space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
        log_msg = self.delimiter.join([
            header,
            '[{0' + space_fmt + '}/{1}]',
            'eta: {eta}',
            '{meters}',
            'time: {time}',
            'data: {data}',
            'max mem: {memory:.0f}'
        ])
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                print(log_msg.format(
                    i, len(iterable), eta=eta_string,
                    meters=str(self),
                    time=str(iter_time), data=str(data_time),
                    memory=torch.cuda.max_memory_allocated() / MB))
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print('{} Total time: {} ({:.4f} s / it)'.format(
            header, total_time_str, total_time / len(iterable)))


def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):

    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha

    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)


def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__
    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0

def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)

def init_distributed_mode(args):
    if 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ['WORLD_SIZE'])
        args.gpu = int(os.environ['LOCAL_RANK'])
    elif 'SLURM_PROCID' in os.environ:
        args.rank = int(os.environ['SLURM_PROCID'])
        args.gpu = args.rank % torch.cuda.device_count()
    else:
        print('Not using distributed mode')
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = 'nccl'
    print('| distributed init (rank {}): {}'.format(
        args.rank, args.dist_url), flush=True)
    torch.distributed.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                         world_size=args.world_size, rank=args.rank)
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has 21 classes, 20 items and 1 background class
num_classes = len(CLASSES)

# get the model using our helper function
model = get_model(num_classes)
# move model to the either CPU or GPU - GPU is recommended
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad] # The optimizer requires the parameters of the model
# to be passed in.

optimizer = torch.optim.SGD(params, lr=0.005, # Using Stochastic Gradient Descent as our optimizer
                            momentum=0.9, weight_decay=0.0005)

# Learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


In [0]:
import math

In [0]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train() # Setting the model to training mode.
    metric_logger = MetricLogger(delimiter="  ") # This is a utility provided by the Torchvision tutorial
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}')) # We are taking advantage of it.
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)
    
    ## Typical Training loop ##
    for images, targets in metric_logger.log_every(data_loader, print_freq, header=header):
        
        images = list(image.to(device) for image in images)
        targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]
        
        ## Getting the outputs
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)
        
        ## Resetting the Gradients ##
        optimizer.zero_grad()
        ## Backpropagation ##
        losses.backward()
        ## Updating the weights ##
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()
        
        # For logging
        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

In [0]:
# Training for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: [0]  [  0/732]  eta: 0:20:52  lr: 0.000012  loss: 4.2840 (4.2840)  loss_classifier: 2.9888 (2.9888)  loss_box_reg: 0.1629 (0.1629)  loss_mask: 1.0780 (1.0780)  loss_objectness: 0.0063 (0.0063)  loss_rpn_box_reg: 0.0480 (0.0480)  time: 1.7107  data: 0.2422  max mem: 3115
Epoch: [0]  [ 10/732]  eta: 0:06:07  lr: 0.000080  loss: 4.5183 (4.6226)  loss_classifier: 2.9734 (2.9412)  loss_box_reg: 0.2211 (0.2164)  loss_mask: 1.2351 (1.4203)  loss_objectness: 0.0118 (0.0135)  loss_rpn_box_reg: 0.0164 (0.0312)  time: 0.5087  data: 0.0281  max mem: 3410
Epoch: [0]  [ 20/732]  eta: 0:05:25  lr: 0.000148  loss: 3.9940 (3.8943)  loss_classifier: 2.3729 (2.3270)  loss_box_reg: 0.1860 (0.1902)  loss_mask: 1.2351 (1.3409)  loss_objectness: 0.0092 (0.0119)  loss_rpn_box_reg: 0.0124 (0.0244)  time: 0.3948  data: 0.0065  max mem: 3726
Epoch: [0]  [ 30/732]  eta: 0:05:08  lr: 0.000217  loss: 2.2239 (3.2726)  loss_classifier: 0.8993 (1.7731)  loss_box_reg: 0.1913 (0.2137)  loss_mask: 0.9946 (1.2519) 